In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd
import xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Read Data
path = '/content/drive/MyDrive/NLP/Sentimental Analysis Project/Amazon_Customer_Reviews.csv'
data = pd.read_csv(path)
data.drop('Unnamed: 0',axis=1,inplace=True)
data.head()


,Ratings,Reviews
0,Good,"I use Amazon.com often, and 99% of the time, ..."
1,Good,With Amazon you can leisurely shop in the com...
2,Good,I am very happy with all my purchases since I...
3,Good,"I absolutely love Amazon. Their selections, pr..."
4,Good,We have shopped Amazon for years and always fi...


In [6]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [7]:
#Data cleaning and preprocessing
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wordnet = WordNetLemmatizer()

# Function for preprocessing
def preprocessing(data):    
    
    reviews =  []
    for i in range(0, len(data)):
        # Replacing values in rows
        review = data['Reviews'][i]
        review = review.replace('$','dollars')
    
        # keeping only text and numbers in reviews
        review = re.sub('%', ' percent', review)
        review = re.sub('[^a-zA-Z0-9/]', ' ', review)
        review = review.lower()
        review = review.split()
    
        # Removing Stopwords
        review = [wordnet.lemmatize(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
    
        reviews.append(review)
    
    return reviews
        
reviews_1 = preprocessing(data)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<h3>Bag of words</h3>

In [37]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(reviews_1).toarray()

#dummyfying output variable
y=pd.get_dummies(data['Ratings'],drop_first=True)

<h3>TF-IDF</h3>

In [27]:
# TFIDF using TFIDFVectorizer    
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(max_features=5000)
X = cv.fit_transform(reviews_1).toarray()

#dummyfying output variable
y=pd.get_dummies(data['Ratings'],drop_first=True)

In [28]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)




<h3>Aritificial Neural Network</h3>

In [29]:
from tensorflow.keras.layers import Bidirectional
## Creating model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

model=Sequential()
model.add(Dense(units = 1000, kernel_initializer=  'uniform', activation = 'relu', input_dim = 5000))
model.add(Dropout(0.3))
model.add(Dense(units = 250, kernel_initializer=  'uniform', activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 100, kernel_initializer=  'uniform', activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 25, kernel_initializer=  'uniform', activation = 'relu'))
model.add(Dense(1,kernel_initializer= 'uniform',activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())



Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 1000)              5001000   
_________________________________________________________________
dropout_9 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 250)               250250    
_________________________________________________________________
dropout_10 (Dropout)         (None, 250)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 100)               25100     
_________________________________________________________________
dropout_11 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 25)               

In [30]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
83/83 [==============================] - 5s 55ms/step - loss: 0.5667 - accuracy: 0.6977 - val_loss: 0.3084 - val_accuracy: 0.9381
Epoch 2/10
83/83 [==============================] - 4s 51ms/step - loss: 0.2312 - accuracy: 0.9527 - val_loss: 0.1843 - val_accuracy: 0.9419
Epoch 3/10
83/83 [==============================] - 4s 51ms/step - loss: 0.0820 - accuracy: 0.9789 - val_loss: 0.2361 - val_accuracy: 0.9426
Epoch 4/10
83/83 [==============================] - 4s 51ms/step - loss: 0.0354 - accuracy: 0.9908 - val_loss: 0.3072 - val_accuracy: 0.9396
Epoch 5/10
83/83 [==============================] - 4s 51ms/step - loss: 0.0190 - accuracy: 0.9946 - val_loss: 0.3927 - val_accuracy: 0.9396
Epoch 6/10
83/83 [==============================] - 4s 51ms/step - loss: 0.0081 - accuracy: 0.9977 - val_loss: 0.4915 - val_accuracy: 0.9336
Epoch 7/10
83/83 [==============================] - 4s 51ms/step - loss: 0.0054 - accuracy: 0.9986 - val_loss: 0.5548 - val_accuracy: 0.9343
Epoch 8/10
83

In [31]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)

In [32]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[922  24]
 [ 56 323]]


In [33]:
# Classification metrics
from sklearn.metrics import accuracy_score
result2 = accuracy_score(y_test,y_pred)
print("Accuracy:",result2)

Accuracy: 0.939622641509434


In [47]:
re = ['nice product']
cv = CountVectorizer(max_features=5000)
test = cv.fit_transform(re).toarray()

pred = model.predict(test)

print(pred)

InvalidArgumentError: ignored

In [46]:
test



array([[0, 1, 1, 0],
       [1, 0, 0, 1]])